In [1]:
import pandas as pd
import glob
import os

# set the working directory to current one
os.chdir('/mnt/c/Users/User/notebook/jupyterenv/generic-real-estate-consulting-project-18/')


# set the path to load the crime files
# creates a files variable with glob, this will be used to load the files
path = "data/raw/crime/2022/csv/"

files = glob.glob(path + "/*.csv")


# this list is to store the files as dataframes in a list
dataframes_list = []


# process of reading and storing
for i in range(len(files)):
    temp_df = pd.read_csv(files[i])
    dataframes_list.append(temp_df)

    
# new list to store cleaned dataframes
cleaned_dataframes = []

# cleaning the columns of the files since they all have similar columns
# since family incidents only records from 2018 we do the same for the rest
# remove the total columns as they seem irrelevant

for df in dataframes_list:
    df.dropna(inplace=True)
    df['Year'] = df["Year"].astype('int')
    df_new = df[df['Year'] >= 2018]
    df_new = df_new.loc[df['Local Government Area'] != 'Total']
    cleaned_dataframes.append(df_new)
    



In [2]:
# assign variable names to each dataframe then rename every rate per population column


alleged_offenders = cleaned_dataframes[0]

criminal_incidents = cleaned_dataframes[1]

family_incidents = cleaned_dataframes[2]

recorded_offences = cleaned_dataframes[3]

victim_reports = cleaned_dataframes[4]

alleged_offenders.rename(columns = {'Rate per 100,000 population':'per_100,000_alleged_off'}, inplace = True)

criminal_incidents.rename(columns = {'Rate per 100,000 population':'per_100,000_crim_incidents'}, inplace = True)

family_incidents.rename(columns = {'Rate per 100,000 population':'per_100,000_fam_incidents'}, inplace = True)

recorded_offences.rename(columns = {'Rate per 100,000 population':'per_100,000_recorded_off'}, inplace = True)

victim_reports.rename(columns = {'Rate per 100,000 population':'per_100,000_vic_reports'}, inplace = True)

In [3]:
# merge the dataframes into one big one


merged_dataframes_1 = pd.merge(pd.merge(alleged_offenders,criminal_incidents,on=['Year', 'Local Government Area'], 
                                        how = 'outer'), family_incidents, on=['Year', 'Local Government Area'], 
                                        how = 'outer', suffixes=('', '_y', '_x'))
                              

final_df = pd.merge(pd.merge(merged_dataframes_1,recorded_offences,on=['Year', 'Local Government Area'], 
                                        how = 'outer'), victim_reports, on=['Year', 'Local Government Area'], 
                                        how = 'outer', suffixes=('', '_y', '_x'))

final_df.drop(final_df.filter(regex='_y$').columns, axis=1, inplace=True)

final_df.drop(final_df.filter(regex='_x$').columns, axis=1, inplace=True)

final_df.rename(columns = {'Local Government Area':'LGA code'}, inplace = True)

final_df

/tmp/ipykernel_149/2975143067.py:9: FutureWarning: Passing 'suffixes' which cause duplicate columns {'Police Region_x', 'Year ending_x'} in the result is deprecated and will raise a MergeError in a future version.
  final_df = pd.merge(pd.merge(merged_dataframes_1,recorded_offences,on=['Year', 'Local Government Area'],


,Year,LGA code,Alleged Offender Incidents,"per_100,000_alleged_off",Incidents Recorded,"per_100,000_crim_incidents",Family Incidents,"per_100,000_fam_incidents",Offence Count,"per_100,000_recorded_off",Year ending,Police Region,Victim Reports,"per_100,000_vic_reports"
0,2022,Banyule,"2,547","1,985.0","5,244","4,086.9","1,185",923.5,"7,191","5,604.2",March,1 North West Metro,"2,826","2,202.4"
1,2022,Brimbank,"5,062","2,545.8","12,330","6,201.0","2,776","1,396.1","16,839","8,468.6",March,1 North West Metro,"7,056","3,548.6"
2,2022,Darebin,"4,084","2,532.2","9,276","5,751.5","1,610",998.3,"12,424","7,703.4",March,1 North West Metro,"5,197","3,222.3"
3,2022,Hobsons Bay,"2,005","2,094.8","4,599","4,804.9","1,031","1,077.2","5,884","6,147.4",March,1 North West Metro,"2,679","2,798.9"
4,2022,Hume,"5,915","2,386.0","12,196","4,919.7","3,850","1,553.0","17,870","7,208.5",March,1 North West Metro,"7,056","2,846.3"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390,2018,Surf Coast,352,"1,091.4",952,"2,951.8",177,548.8,"1,142","3,540.9",March,4 Western,657,"2,037.1"
391,2018,Swan Hill,"1,078","5,192.7","1,752","8,439.3",609,"2,933.5","2,466","11,878.6",March,4 Western,924,"4,450.9"
392,2018,Warrnambool,"1,597","4,580.8","2,707","7,764.7",627,"1,798.5","3,555","10,197.1",March,4 Western,"1,230","3,528.1"
393,2018,West Wimmera,60,"1,553.6",114,"2,951.8",41,"1,061.6",161,"4,168.8",March,4 Western,70,"1,812.5"


In [4]:
# save the dataframe

final_df.to_csv("data/curated/crime/LGA_crime_clean.csv")